In [ ]:
pip install tokenizers

     |████████████████████████████████| 6.6 MB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
my_vocab_size = 32000
# vocab의 크기를 의미한다. 적을 수록 "단어" 단위로, 클 수록 "음절" 단위로 나뉘어진다.
my_limit_alphabet = 6000
# 모든 알파벳을 커버할 수 있도록 하여, [UNK] 빈도를 줄이기 위해 6000을 선택했다.
my_special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# tokenizer 에서 사용될 special_tokens이다. 필수 토큰은 위와 같다.
user_defined_symbols = ['[BOS]','[EOS]']
# 이제부터는 부가적인 토큰이다. 문장의 시작과 끝을 알리는 토큰을 추가했다.
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
# KoELECTRA Github를 참고하여, unused 토큰을 약 200개 추가했다. 범용성을 높일 수 있다.
user_defined_symbols = user_defined_symbols + unused_list
my_special_tokens = my_special_tokens + user_defined_symbols

In [32]:
from pathlib import Path
from tokenizers import BertWordPieceTokenizer

paths = [str(x) for x in Path("./data/train").glob("sqlCorpus.txt")]
# 학습에 사용될 Corpus들을 넣으면 된다. 

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True, 
    # 만약 cased model이라면 반드시 False로 해야한다, 또한 한글의 경우 cased model로 하면 글자가 자소분리된다.
    lowercase=True,
    # 대소문자 구분 여부를 의미한다. 한글의 경우 무의미하므로 신경쓰지 않아도 된다.
    wordpieces_prefix="##"
)

tokenizer.train(
    files=paths,
    limit_alphabet=my_limit_alphabet,
    vocab_size=my_vocab_size,
    min_frequency=5,
    # pair가 5회이상 등장할시에만 학습
    show_progress=True,
    # 진행과정 출력 여부
    special_tokens=my_special_tokens
)

In [45]:
tokenizer.save("./resultVocabJson.json".format(my_limit_alphabet, my_vocab_size),True)
# config, model등 저장하지 않고 vocab 정보만 json 형태로 저장



import json
vocab_path = "./resultVocabJson.json"
# save의 결과로 추출된 파일 경로
vocab_file = './resultVocabTxt.txt'
# vocab.txt 형태로 저장할 경로
f = open(vocab_file,'w',encoding='utf-8')
with open(vocab_path) as json_file:
    json_data = json.load(json_file)
    for item in json_data["model"]["vocab"].keys():
        f.write(item+'\n')

    f.close()